# Pre-process_Xenium_Prime_Human_Skin_FFPE_outs


In [1]:
%load_ext autoreload
%autoreload 2
%env ANYWIDGET_HMR=1

env: ANYWIDGET_HMR=1


In [2]:
import numpy as np
import pandas as pd

# macOS requirement
import os
os.environ['DYLD_LIBRARY_PATH'] = '/opt/homebrew/lib:' + os.environ.get('DYLD_LIBRARY_PATH', '')

import celldega as dega

import tifffile
import zarr

import matplotlib.pyplot as plt
from matplotlib.colors import to_hex

import geopandas as gpd
import shapely

import tarfile

In [3]:
dataset_name = 'Xenium_Prime_Human_Skin_FFPE_outs'

In [4]:
base_path = 'data/xenium_data/' + dataset_name + '/'

In [5]:
path_landscape_files = 'data/xenium_landscapes/' + dataset_name + '/'

In [6]:
base_path

'data/xenium_data/Xenium_Prime_Human_Skin_FFPE_outs/'

In [7]:
path_landscape_files

'data/xenium_landscapes/Xenium_Prime_Human_Skin_FFPE_outs/'

In [8]:
if not os.path.exists(path_landscape_files):
    os.mkdir(path_landscape_files)

## Unzip Xenium Data

#### Decompress Cell Feature Matrix MTX Files

In [9]:
# Path to the tar.gz file you want to decompress
tar_file_path = base_path + 'cell_feature_matrix.tar.gz'
# Path to the directory where you want to extract the contents
output_directory = path_landscape_files

# Open the tar.gz file
with tarfile.open(tar_file_path, "r:gz") as tar:
    # Extract all contents to the specified directory
    tar.extractall(path=output_directory)

print(f"File {tar_file_path} has been decompressed to {output_directory}")


File data/xenium_data/Xenium_Prime_Human_Skin_FFPE_outs/cell_feature_matrix.tar.gz has been decompressed to data/xenium_landscapes/Xenium_Prime_Human_Skin_FFPE_outs/


#### Decompress Xenium Analysis Files

In [10]:
# Path to the tar.gz file you want to decompress
tar_file_path = base_path + 'analysis.tar.gz'
# Path to the directory where you want to extract the contents
output_directory = path_landscape_files

# Open the tar.gz file
with tarfile.open(tar_file_path, "r:gz") as tar:
    # Extract all contents to the specified directory
    tar.extractall(path=output_directory)

print(f"File {tar_file_path} has been decompressed to {output_directory}")


File data/xenium_data/Xenium_Prime_Human_Skin_FFPE_outs/analysis.tar.gz has been decompressed to data/xenium_landscapes/Xenium_Prime_Human_Skin_FFPE_outs/


# CBG

In [11]:
cbg = dega.pre.read_cbg_mtx(path_landscape_files + 'cell_feature_matrix/')
cbg

read mtx file from  data/xenium_landscapes/Xenium_Prime_Human_Skin_FFPE_outs/cell_feature_matrix/


1,A2ML1,AAMP,AAR2,AARSD1,ABAT,ABCA1,ABCA3,ABCA4,ABCA7,ABCB1,...,DeprecatedCodeword_17695,DeprecatedCodeword_17827,DeprecatedCodeword_18021,DeprecatedCodeword_18190,DeprecatedCodeword_18274,DeprecatedCodeword_18325,DeprecatedCodeword_18378,DeprecatedCodeword_18394,DeprecatedCodeword_18534,DeprecatedCodeword_18649
0,,,,,,,,,,,,,,,,,,,,,
aaaaljij-1,18,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
aaabgfcl-1,24,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
aaacghkb-1,28,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
aaachnfg-1,14,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
aaacknep-1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
oinnidan-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
oinpfcne-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
oinphhmf-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
meta_gene_exp = dega.pre.calc_meta_gene_data(cbg)

calculating mean expression from sparse float data
calculating variance by looping over rows


### Gene Metadata

In [13]:
path_cbg = path_landscape_files + 'cell_feature_matrix/'
path_output = path_landscape_files + 'meta_gene.parquet'
dega.pre.make_meta_gene('Xenium', path_cbg, path_output)

read mtx file from  data/xenium_landscapes/Xenium_Prime_Human_Skin_FFPE_outs/cell_feature_matrix/
calculating mean expression from sparse float data
calculating variance by looping over rows


### Cell-by-gene Files

In [14]:
dega.pre.save_cbg_gene_parquets(path_landscape_files, cbg, verbose=True)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000


### Image Tiles

In [15]:
import tifffile

# Path to your OME-TIFF file
file_path = base_path + 'morphology_focus/morphology_focus_0000.ome.tif'

# Open the OME-TIFF file and read the image data
with tifffile.TiffFile(file_path) as tif:
    series = tif.series[0]  # Assuming you are interested in the first series
    image_data = series.asarray()


<tifffile.TiffFile 'morphology_focus_0000.ome.tif'> OME series cannot read multi-file pyramids


In [16]:
image_data.shape

(4, 20294, 42748)

### DAPI

In [20]:
image_scale = 1

In [21]:
image_data_scaled = image_data[0,:,:] * 2

# Save the image data to a regular TIFF file without compression
tifffile.imwrite(path_landscape_files + 'output_regular.tif', image_data_scaled, compression=None)
image_ds = dega.pre.reduce_image_size(path_landscape_files + 'output_regular.tif', image_scale, path_landscape_files)
image_png = dega.pre.convert_to_png(image_ds)
dega.pre.make_deepzoom_pyramid(image_png, path_landscape_files + 'pyramid_images/', 'dapi', suffix=".webp[Q=100]")

In [22]:
image_data_scaled = image_data[1,:,:] * 2

# Save the image data to a regular TIFF file without compression
tifffile.imwrite(path_landscape_files + 'output_regular.tif', image_data_scaled, compression=None)
image_ds = dega.pre.reduce_image_size(path_landscape_files + 'output_regular.tif', image_scale, path_landscape_files)
image_png = dega.pre.convert_to_png(image_ds)
dega.pre.make_deepzoom_pyramid(image_png, path_landscape_files + 'pyramid_images/', 'bound', suffix=".webp[Q=100]")

In [23]:
image_data_scaled = image_data[2,:,:] * 2

# Save the image data to a regular TIFF file without compression
tifffile.imwrite(path_landscape_files + 'output_regular.tif', image_data_scaled, compression=None)
image_ds = dega.pre.reduce_image_size(path_landscape_files + 'output_regular.tif', image_scale, path_landscape_files)
image_png = dega.pre.convert_to_png(image_ds)
dega.pre.make_deepzoom_pyramid(image_png, path_landscape_files + 'pyramid_images/', 'rna', suffix=".webp[Q=100]")

In [24]:
image_data_scaled = image_data[3,:,:] * 2

# Save the image data to a regular TIFF file without compression
tifffile.imwrite(path_landscape_files + 'output_regular.tif', image_data_scaled, compression=None)
image_ds = dega.pre.reduce_image_size(path_landscape_files + 'output_regular.tif', image_scale, path_landscape_files)
image_png = dega.pre.convert_to_png(image_ds)
dega.pre.make_deepzoom_pyramid(image_png, path_landscape_files + 'pyramid_images/', 'prot', suffix=".webp[Q=100]")

### Cell Metadata

In [25]:
# Function to open a Zarr file
def open_zarr(path: str) -> zarr.Group:
    store = (zarr.ZipStore(path, mode="r")
    if path.endswith(".zip")
    else zarr.DirectoryStore(path)
    )
    return zarr.group(store=store)

# For example, use the above function to open the cells Zarr file, which contains segmentation mask Zarr arrays
root = open_zarr(base_path + "cells.zarr.zip")

# # Look at group array info and structure
# root.info
# root.tree() # shows structure, array dimensions, data types


In [26]:
transformation_matrix = root['masks']['homogeneous_transform'][:]
transformation_matrix

array([[4.705882, 0.      , 0.      , 0.      ],
       [0.      , 4.705882, 0.      , 0.      ],
       [0.      , 0.      , 1.      , 0.      ],
       [0.      , 0.      , 0.      , 1.      ]], dtype=float32)

In [27]:
pd.DataFrame(transformation_matrix[:3,:3]).to_csv(
    path_landscape_files + 'xenium_transform.csv', 
    sep=' ', 
    header=False, 
    index=False
)

In [28]:
path_transformation_matrix = path_landscape_files + 'xenium_transform.csv'
path_meta_cell_micron = base_path + 'cells.csv.gz'
path_meta_cell_image = path_landscape_files + 'cell_metadata.parquet'

In [29]:
default_clustering = pd.read_csv(path_landscape_files + 'analysis/clustering/gene_expression_graphclust/clusters.csv', index_col=0)
default_clustering

,Cluster
Barcode,
aaaaljij-1,28
aaabgfcl-1,27
aaacghkb-1,27
aaachnfg-1,28
aaacknep-1,28
...,...
oinnidan-1,5
oinpfcne-1,5
oinphhmf-1,5


### Save cell metadata

In [34]:
# # do not including clustering information in default cell metadata
# dega.pre.make_meta_cell_image_coord(
#     'Xenium', 
#     path_transformation_matrix, 
#     path_meta_cell_micron, 
#     path_meta_cell_image, 
#     image_scale
# )

In [35]:
# df_meta = pd.read_csv(path_landscape_files + 'analysis/clustering/gene_expression_graphclust/clusters.csv', index_col=0)
# df_meta['Cluster'] = df_meta['Cluster'].astype('string')
# df_meta.columns = ['cluster']

In [37]:
dega.pre.make_meta_cell_image_coord(
    'Xenium', 
    path_transformation_matrix, 
    path_meta_cell_micron, 
    path_meta_cell_image, 
    image_scale=image_scale
)

### Save default clustering results

In [38]:
if not os.path.exists(path_landscape_files + 'cell_clusters/'):
    os.mkdir(path_landscape_files + 'cell_clusters/')

In [39]:
default_clustering = pd.DataFrame(default_clustering.values, index=default_clustering.index.tolist(), columns=['cluster'])
default_clustering.head()

,cluster
aaaaljij-1,28
aaabgfcl-1,27
aaacghkb-1,27
aaachnfg-1,28
aaacknep-1,28


In [40]:
default_clustering_ini = pd.DataFrame(default_clustering.values, index=default_clustering.index.tolist(), columns=['cluster'])
default_clustering_ini.head()

,cluster
aaaaljij-1,28
aaabgfcl-1,27
aaacghkb-1,27
aaachnfg-1,28
aaacknep-1,28


In [41]:
meta_cell = pd.read_parquet(path_landscape_files + 'cell_metadata.parquet')
meta_cell.shape

(112551, 2)

In [42]:
default_clustering_ini['cluster'] = default_clustering_ini['cluster'].astype('string')

In [43]:
default_clustering = pd.DataFrame(index=meta_cell.index.tolist())

default_clustering.loc[default_clustering_ini.index.tolist(), 'cluster'] = default_clustering_ini['cluster']

In [44]:
default_clustering.to_parquet(path_landscape_files + 'cell_clusters/cluster.parquet')

### Cluster Colors

In [45]:
ser_counts = default_clustering['cluster'].value_counts()
clusters = ser_counts.index.tolist()

In [46]:
# Get all categorical color palettes from Matplotlib and flatten them into a single list of colors
palettes = [plt.get_cmap(name).colors for name in plt.colormaps() if "tab" in name]
flat_colors = [color for palette in palettes for color in palette]

# Convert RGB tuples to hex codes
flat_colors_hex = [to_hex(color) for color in flat_colors]

# Use modular arithmetic to assign a color to each gene, white for genes with "Blank"
colors = [
    flat_colors_hex[i % len(flat_colors_hex)] if "Blank" not in cluster else "#FFFFFF"
    for i, cluster in enumerate(clusters)
]

# Create a DataFrame with genes and their assigned colors
ser_color = pd.Series(colors, index=clusters, name='color')

meta_cluster = pd.DataFrame(ser_color)

meta_cluster['count'] = ser_counts

meta_cluster.to_parquet(path_landscape_files + 'cell_clusters/meta_cluster.parquet')

### Transcripts

In [ ]:
%%time 
technology = 'Xenium'
path_trx = base_path + 'transcripts.parquet'
path_trx_tiles = path_landscape_files + 'transcript_tiles'
tile_bounds = dega.pre.make_trx_tiles(
    'Xenium', 
    path_trx, 
    path_transformation_matrix, 
    path_trx_tiles,
    tile_size=100,
    # verbose=True
    image_scale=image_scale
)

### Cell Boundaries

In [ ]:
%%time
path_cell_boundaries = base_path + 'cell_boundaries.parquet'
path_output = path_landscape_files + 'cell_segmentation'
dega.pre.make_cell_boundary_tiles(
    'Xenium',
    path_cell_boundaries, 
    path_meta_cell_micron, 
    path_transformation_matrix, 
    path_output,
    tile_size=100,
    tile_bounds=tile_bounds,
    image_scale=image_scale
)

### Gene Metadata

In [ ]:
path_cbg = path_landscape_files + 'cell_feature_matrix/'
path_output = path_landscape_files + 'gene_metadata.parquet'
dega.pre.make_meta_gene('Xenium', path_cbg, path_output)

### Max Zoom

In [ ]:
# Example usage:
path_image_pyramid = path_landscape_files + 'pyramid_images/dapi_files/'  # Change this to your actual directory path
max_pyramid_zoom = dega.pre.get_max_zoom_level(path_image_pyramid)

print(max_pyramid_zoom)

### Cluster Gene Expression

In [ ]:
usecols = ['cell_id', 'x_centroid', 'y_centroid']
meta_cell = pd.read_csv(base_path + 'cells.csv.gz', index_col=0, usecols=usecols)
meta_cell.columns = ['center_x', 'center_y']
meta_cell

In [ ]:
meta_cell['cluster'] = df_meta['cluster']

In [ ]:
list_ser = []
for inst_cat in meta_cell['cluster'].unique().tolist():
    if inst_cat is not None:
        inst_cells = meta_cell[meta_cell['cluster'] == inst_cat].index.tolist()
        # print(inst_cat, len(inst_cells))

        inst_ser = cbg.loc[inst_cells].sum()/len(inst_cells)
        inst_ser.name = inst_cat

        list_ser.append(inst_ser)

df_sig = pd.concat(list_ser, axis=1)    


In [ ]:
df_sig = pd.concat(list_ser, axis=1)
# handling weird behavior where there is a multiindex it appears
df_sig.columns = df_sig.columns.tolist()
df_sig.index = df_sig.index.tolist()

In [ ]:
keep_genes = df_sig.index.tolist()
keep_genes = [x for x in keep_genes if 'Unassigned' not in x]
keep_genes = [x for x in keep_genes if 'NegControl' not in x]
keep_genes = [x for x in keep_genes if 'DeprecatedCodeword' not in x]
len(keep_genes)

df_sig = df_sig.loc[keep_genes, clusters]
df_sig.shape

In [ ]:
df_sig.sparse.to_dense().to_parquet(path_landscape_files + 'df_sig.parquet')

### Save Landscape Parameters JSON

In [ ]:
image_info =  [
        {
            "name": "dapi",
            "button_name": "DAPI",
            "color": [
                0,
                0,
                255
            ]
        },
        {
            "name": "bound",
            "button_name": "BOUND",
            "color": [
                0,
                255,
                0
            ]
        },
        {
            "name": "rna",
            "button_name": "RNA",
            "color": [
                255,
                0,
                0
            ]
        },
        {
            "name": "prot",
            "button_name": "PROT",
            "color": [
                255,
                255,
                255
            ]
        }
    ]

In [ ]:
dega.pre.save_landscape_parameters(
    'Xenium', 
    path_landscape_files,
    'dapi_files',
    tile_size=100,
    image_info=image_info,
    image_format='.webp'
)